#### Exercise 1
<!-- @q -->

1. What kinds of EDA techniques might you use to explore the following types of data:
    - Numeric data?  
    With numeric data, I might use a dunction like `describe()` to produce summary statistics on any and all variables I plan to use (exclude row counts, etc.). I'll also want to inspect the distribution of my target variable(s) using a histogram.
    - Categorical data?  
    Similarly, I will want to get a feel for the categorical data. I can use barplots to inspect the distribution, and run summary statistics to understand the mode(s) and uncover any ranked variables.
    - The relationship between categorical and numeric data?
    If we're just running EDA, the approach would simply be to inspect the numeric data in its own way and the categprical in its own. One should also look at encoding the variables into numeric bins, or other ways to get them to interact.

*Enter your answer in this cell*

2. Generate some fake data (~1000 rows) with 1 categorical column (with 10 categories) and 2 numeric columns. Use the techniques you mentioned to explore the numeric, categorical, and the relationship between them.

In [7]:
# Your code here
import random
import pandas as pd
one_to_ten = [random.randint(1, 10) for i in range(1000)]
two_to_twelve = [random.randint(2, 12) for i in range(1000)]
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet', 'white', 'black', 'gold']
colors_list = [random.choice(colors) for i in range(1000)]
df = pd.DataFrame({
    "one_to_ten": one_to_ten,
    "two_to_twelve": two_to_twelve,
    "colors": colors_list
})

In [8]:
# Your code here
df.head()

,one_to_ten,two_to_twelve,colors
0,3,9,blue
1,10,7,violet
2,10,3,green
3,9,10,orange
4,1,9,white


In [12]:
# Your code here
df.describe()

,one_to_ten,two_to_twelve
count,1000.000000,1000.000000
mean,5.433000,6.663000
std,2.911011,3.156802
min,1.000000,2.000000
25%,3.000000,4.000000
50%,5.000000,7.000000
75%,8.000000,9.000000
max,10.000000,12.000000


In [14]:
# Your code here
df['colors'].value_counts()

colors
violet    108
white     108
yellow    108
black     105
blue       99
indigo     99
red        98
gold       95
green      93
orange     87
Name: count, dtype: int64

In [17]:
# Your code here
df["colors_encoded"] = df["colors"].astype("category").cat.codes
#Code from ChatGPT for .cat and .codes
df.head()

,one_to_ten,two_to_twelve,colors,colors_encoded
0,3,9,blue,1
1,10,7,violet,7
2,10,3,green,3
3,9,10,orange,5
4,1,9,white,8


In [ ]:
df['colors_encoded'].describe()
#If my colors variable was ranked or something

count    1000.000000
mean        4.579000
std         2.931954
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max         9.000000
Name: colors_encoded, dtype: float64

#### Exercise 2


Generate a data set you can use with a supervised ML model.  The data should meet the following criteria:
   - It should have 1000 rows
   - It should have 6 columns, with one column (your "target" column being a boolean column), one categorical column with 5 categories, and 4 numeric columns.
   - The numeric columns should have dramatically different scales - different means, different std. deviations.
   - Each non-target column should have about 5% nulls.

Make this data a little more interesting by calculating the target column using a noisy function of the other columns.

In [5]:
# Your code here
import numpy as np
import pandas as pd

num1 = np.random.normal(loc=10, scale=2, size=1000)
num2 = np.random.normal(loc=500, scale=50, size=1000)
num3 = np.random.normal(loc=1000, scale=2000, size=1000)
num4 = np.random.normal(loc=1, scale=.1, size=1000)

# categorical (5 levels) — note: no extra brackets
colleges = ['syracuse', 'buffalo', 'binghamton', 'cortland', 'colgate']
cat_col = np.random.choice(colleges, 1000)

# noisy target = function(num1..4, category) + noise, threshold at 0
cat_weights = {'syracuse':0.6,'buffalo':0.2,'binghamton':0.0,'cortland':-0.2,'colgate':-0.4}
signal = (0.5*num1 + 0.01*num2 - 0.00005*num3 + 2*num4
          + np.vectorize(cat_weights.get)(cat_col))
score = signal + np.random.normal(0,1,1000)
target = (score > np.median(score)).astype(int)

df = pd.DataFrame({
    'target': target,
    'category': cat_col,
    'num1': num1, 'num2': num2, 'num3': num3, 'num4': num4
})

# null adder taken from internet
for c in ['category','num1','num2','num3','num4']:
    idx = np.random.choice(df.index, int(0.05*1000), replace=False)
    df.loc[idx, c] = np.nan

print(df.head())
print(df.isna().mean())

   target  category       num1        num2         num3      num4
0       0  cortland        NaN  426.252585  2558.128402  1.040625
1       1   colgate        NaN  572.988808   277.922537  0.873543
2       0   colgate   8.414419  436.367895   211.262391  0.952254
3       0  cortland  10.880240         NaN -1305.948431  1.163166
4       0  cortland   9.662175  477.079163  -326.668827  0.927046
target      0.00
category    0.05
num1        0.05
num2        0.05
num3        0.05
num4        0.05
dtype: float64


#### Exercise 3

Use whatever resources you need to figure out how to build an SKLearn ML pipelines. Use a pipeline to build an ML approach to predicting your target column in the preceding data with logistic regression.  I have set up the problem below so that you will write your code in a function function call that takes an SKLearn model and data frame and returns the results of a cross validation scoring routine.  

I have not taught you how to do this; use the book, google, the notes, chatgpt, or whatever. This is a test of your ability to *find* information, and use this to construct a solution. Your solution should:

- Use a transformer pipeline that processes your numeric and categorical features separately
- Place everything in a pipeline with the classifier that is passed in to the function.
- I've already implemented the call to cross_val_score - to make it work, you'll need to assign your pipeline to the `pipeline` variable.

_Note: You could just feed this question to AI and get an answer, and chances are, it will be right. But if you do, you won't really learn much. So, be thoughtful in your use of AI here - you can use it to build the solution step by step, and it will explain how everything works. It's all in how you use it. So, it's your choice - go for the easy grade, or learn something._

In [6]:
# --- Imports
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def run_classifier(df,classifier):
    # Separate features/target
    y = df["target"].astype(int)  # this is redundant? eh why not
    X = df.drop(columns=["target"])

    #You fill in the pipeline definition.  Make sure to:
    # - process categorical features (using an imputer and one hot encoder)
    # - process numeric features (using an imputer and StandardScaler)
    # - define your pipeline using `pipeline = ...`
    
    num_cols = ["num1", "num2", "num3", "num4"]
    cat_cols = ["category"]

    num_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ])
    cat_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ])

    preprocessor = ColumnTransformer(transformers=[
        ("num", num_transformer, num_cols),
        ("cat", cat_transformer, cat_cols),
    ])

    pipeline = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("clf", classifier),
    ]) 

    # --- 5-fold CV using F1
    return cross_val_score(pipeline, X, y, scoring="f1", cv=5)


scores = run_classifier(df,LogisticRegression(random_state=42))
print(f"F1 (5-fold): mean={scores.mean():.3f}, std={scores.std():.3f}")
print("Fold scores:", np.round(scores, 3))

F1 (5-fold): mean=0.770, std=0.021
Fold scores: [0.738 0.778 0.77  0.761 0.802]


Try using a `RandomForestClassifier` in the preceding pipeline. Just call `run_classifier` with a `RandomForestClassifier`, and print out the results as above.

In [7]:
# Your code here
from sklearn.ensemble import RandomForestClassifier

scores = run_classifier(df, RandomForestClassifier(random_state=42))
print(f"F1 (5-fold): mean={scores.mean():.3f}, std={scores.std():.3f}")
print("Fold scores:", np.round(scores, 3))

F1 (5-fold): mean=0.749, std=0.025
Fold scores: [0.704 0.771 0.759 0.738 0.772]


Normally, `RandomForestClassifier`s are considered to be more powerful than `LogisticRegression`.  Depending on your data, this may or may not be the case. Reflect on your answers - which one does better here, and why do you think that is?  Once again, you might use AI, but you should probably also try to _understand_ the answer.

In this case, the LogisticRegression performs better, despite being the weaker model. If I understand correctly, we put ver simple, linear weights on our variables, which the logistic regression was able to easier detect, and the randomforest likely overcomplicated things. Although I though tree-based was supposed to perform better with categorical and numeric combo models? idk lol

Side note: as i was doing the assignment, I was really confused why the pipelines were useful. I'm used to R and just transforming my dataframe and maybe even making a copy. There are pipelines there, but functionally different. I think after the full assignment, I understand the appeal a little better, especially if the datasets are especially large or you dont want to mess with the underlying data.